In [138]:
import json
import tiktoken
from openai import OpenAI
import yfinance as yf

tokenizer = tiktoken.get_encoding("cl100k_base")
client = OpenAI()

In [139]:
MODEL = "gpt-4o"
CONTEXT_WINDOW = 20000

In [140]:
today = "July 30, 2024"
output_types = """type Info = {
  address1: string;
  city: string;
  state: string;
  zip: string;
  country: string;
  phone: string;
  website: string;
  industry: string;
  industryKey: string;
  industryDisp: string;
  sector: string;
  sectorKey: string;
  sectorDisp: string;
  longBusinessSummary: string;
  fullTimeEmployees: number;
  companyOfficers: {
    maxAge: number;
    name: string;
    age?: number;
    title: string;
    yearBorn?: number;
    fiscalYear: number;
    totalPay?: number;
    exercisedValue: number;
    unexercisedValue: number;
  }[];
  auditRisk: number;
  boardRisk: number;
  compensationRisk: number;
  shareHolderRightsRisk: number;
  overallRisk: number;
  governanceEpochDate: number;
  compensationAsOfEpochDate: number;
  irWebsite: string;
  maxAge: number;
  priceHint: number;
  previousClose: number;
  open: number;
  dayLow: number;
  dayHigh: number;
  regularMarketPreviousClose: number;
  regularMarketOpen: number;
  regularMarketDayLow: number;
  regularMarketDayHigh: number;
  dividendRate: number;
  dividendYield: number;
  exDividendDate: number;
  payoutRatio: number;
  fiveYearAvgDividendYield: number;
  beta: number;
  trailingPE: number;
  forwardPE: number;
  volume: number;
  regularMarketVolume: number;
  averageVolume: number;
  averageVolume10days: number;
  averageDailyVolume10Day: number;
  bid: number;
  ask: number;
  bidSize: number;
  askSize: number;
  marketCap: number;
  fiftyTwoWeekLow: number;
  fiftyTwoWeekHigh: number;
  priceToSalesTrailing12Months: number;
  fiftyDayAverage: number;
  twoHundredDayAverage: number;
  trailingAnnualDividendRate: number;
  trailingAnnualDividendYield: number;
  currency: string;
  enterpriseValue: number;
  profitMargins: number;
  floatShares: number;
  sharesOutstanding: number;
  sharesShort: number;
  sharesShortPriorMonth: number;
  sharesShortPreviousMonthDate: number;
  dateShortInterest: number;
  sharesPercentSharesOut: number;
  heldPercentInsiders: number;
  heldPercentInstitutions: number;
  shortRatio: number;
  shortPercentOfFloat: number;
  impliedSharesOutstanding: number;
  bookValue: number;
  priceToBook: number;
  lastFiscalYearEnd: number;
  nextFiscalYearEnd: number;
  mostRecentQuarter: number;
  earningsQuarterlyGrowth: number;
  netIncomeToCommon: number;
  trailingEps: number;
  forwardEps: number;
  pegRatio: number;
  lastSplitFactor: string;
  lastSplitDate: number;
  enterpriseToRevenue: number;
  enterpriseToEbitda: number;
  '52WeekChange': number;
  'SandP52WeekChange': number;
  lastDividendValue: number;
  lastDividendDate: number;
  exchange: string;
  quoteType: string;
  symbol: string;
  underlyingSymbol: string;
  shortName: string;
  longName: string;
  firstTradeDateEpochUtc: number;
  timeZoneFullName: string;
  timeZoneShortName: string;
  uuid: string;
  messageBoardId: string;
  gmtOffSetMilliseconds: number;
  currentPrice: number;
  targetHighPrice: number;
  targetLowPrice: number;
  targetMeanPrice: number;
  targetMedianPrice: number;
  recommendationMean: number;
  recommendationKey: string;
  numberOfAnalystOpinions: number;
  totalCash: number;
  totalCashPerShare: number;
  ebitda: number;
  totalDebt: number;
  quickRatio: number;
  currentRatio: number;
  totalRevenue: number;
  debtToEquity: number;
  revenuePerShare: number;
  returnOnAssets: number;
  returnOnEquity: number;
  freeCashflow: number;
  operatingCashflow: number;
  earningsGrowth: number;
  revenueGrowth: number;
  grossMargins: number;
  ebitdaMargins: number;
  operatingMargins: number;
  financialCurrency: string;
  trailingPegRatio: number;
};

type PriceData = {
  Open: { [timestamp: string]: number };
  High: { [timestamp: string]: number };
  Low: { [timestamp: string]: number };
  Close: { [timestamp: string]: number };
  Volume: { [timestamp: string]: number };
  Dividends: { [timestamp: string]: number };
  "Stock Splits": { [timestamp: string]: number };
};"""

example0 = """User: Who is the CEO of MSFT?

Assistant: {
  "thought": "Yahoo Finance likely has this information. I'm looking it up...",
  "action": {"command": "get_info", "ticker": "MSFT"}
}

Assistant: {
  "observation": {...,"companyOfficers":[{"maxAge":1,"name":"Mr. Satya  Nadella","age":56,"title":"Chairman & CEO","yearBorn":1967,"fiscalYear":2023,"totalPay":9276400,"exercisedValue":0,"unexercisedValue":0},...],...}
}

Assistant: {
  "answer": "The CEO of MSFT is Satya Nadella."
}

Reflection: I directly retrieved MSFT's company information and answered with the CEO's name. I could have provided more context by explaining the data source and offering additional details about Satya Nadella's tenure or achievements as CEO, if the user wanted more comprehensive information."""
example1 = """User: What was AAPL's highest stock price in the past month, and how does it compare to their 52-week high?

Assistant: {
  "thought": "To answer this question, I need to first get the current stock information for AAPL, including its 52-week high. Then, I'll need to retrieve the stock price history for the past month to find the highest price during that period.",
  "action": {"command": "get_info", "ticker": "AAPL"}
}

Assistant: {
  "observation": {"symbol": "AAPL", "shortName": "Apple Inc.", "fiftyTwoWeekHigh": 198.23, "currency": "USD", ...}
}

Assistant: {
  "thought": "I've got the current stock information for AAPL, including the 52-week high of $198.23. Now I need to get the stock price history for the past month to find the highest price during that period.",
  "action": {"command": "get_history", "ticker": "AAPL", "period": "1mo", "interval": "1d"}
}

Assistant: {
  "observation": {
    "High": {
      "1688407800": 192.67,
      "1688494200": 189.99,
      ...,
      "1690912200": 195.71
    },
    ...
  }
}

Assistant: {
  "thought": "I now have all the information needed to answer the question. Let me analyze the data and provide the answer.",
  "answer": "AAPL's highest stock price in the past month was $195.71. This is 1.27% lower than their 52-week high of $198.23. The highest price in the past month occurred on August 1, 2023 (assuming the last date in the data is the most recent), and it came close to, but did not exceed, the 52-week high."
}

Reflection: I retrieved AAPL's current stock information and 52-week high, then fetched the past month's price history. This two-step approach was thorough, but I could have offered to provide more context about the factors influencing AAPL's recent stock performance if the user desired more in-depth analysis."""

examples = "\n\n".join(
    [
        f"""Example {i}:
```
{example}
```"""
        for i, example in enumerate([example0, example1])
    ]
)

In [141]:
def system_message_prompt(reflections: list[str]) -> str:
    reflections_str = ""
    if len(reflections) > 0:
        reflections_str = "\n\n" + "\n\n".join(reflections)

    return f"""You are a helpful assistant. You run in a loop, seeking additional information to answer a user's question until you are able to answer the question.

Today is {today}.

The output types are:

```typescript
{output_types}
```

The commands to seek information are:

| Command | Arguments | Description | Output Format |
| --- | --- | --- | --- |
| get_info | ticker | Retrieves stock information from Yahoo Finance | Info |
| get_history | ticker, period, interval, start, end | Retrieves stock history from Yahoo Finance | PriceData |

The optional input types are:
- period: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
- interval: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
- start: YYYY-MM-DD
- end: YYYY-MM-DD
You may omit an optional input type if it is not needed. For period and interval the explicit values from the list must be used, for example you cannot use 1wk for the period.

Your response must be in pure JSON format without any additional markers or formatting. The JSON will include a "Thought" + "Action" to retrieve data that you need in order to answer the question, or it will include the "Answer". When data has been retrieved, it will be included as an "Observation".

You will continue generating thoughts and actions until you get to an answer, or conclude that you can't.

IMPORTANT: Ensure that your entire response is valid JSON. Do not include any text outside of the JSON structure, and do not use code block markers or language identifiers. The answer field in the JSON must be a descriptive string, it should not be a JSON.

You may take as many steps as necessary.
Here are some examples.

{examples}

(END OF EXAMPLES){reflections_str}
"""


system_message = system_message_prompt([])


system_message_tokens = len(tokenizer.encode(system_message))
print("Tokens:", system_message_tokens)
print()
print(system_message)

Tokens: 2147

You are a helpful assistant. You run in a loop, seeking additional information to answer a user's question until you are able to answer the question.

Today is July 30, 2024.

The output types are:

```typescript
type Info = {
  address1: string;
  city: string;
  state: string;
  zip: string;
  country: string;
  phone: string;
  website: string;
  industry: string;
  industryKey: string;
  industryDisp: string;
  sector: string;
  sectorKey: string;
  sectorDisp: string;
  longBusinessSummary: string;
  fullTimeEmployees: number;
  companyOfficers: {
    maxAge: number;
    name: string;
    age?: number;
    title: string;
    yearBorn?: number;
    fiscalYear: number;
    totalPay?: number;
    exercisedValue: number;
    unexercisedValue: number;
  }[];
  auditRisk: number;
  boardRisk: number;
  compensationRisk: number;
  shareHolderRightsRisk: number;
  overallRisk: number;
  governanceEpochDate: number;
  compensationAsOfEpochDate: number;
  irWebsite: string;
  

In [142]:
example_previous_trial = [
    {
        "role": "system",
        "content": 'You are a helpful assistant. You run in a loop, seeking additional information to answer a user\'s question until you are able to answer the question.\n\nToday is July 29, 2024.\n\nThe output types are:\n\n```typescript\ntype Info = {\n  address1: string;\n  city: string;\n  state: string;\n  zip: string;\n  country: string;\n  phone: string;\n  website: string;\n  industry: string;\n  industryKey: string;\n  industryDisp: string;\n  sector: string;\n  sectorKey: string;\n  sectorDisp: string;\n  longBusinessSummary: string;\n  fullTimeEmployees: number;\n  companyOfficers: {\n    maxAge: number;\n    name: string;\n    age?: number;\n    title: string;\n    yearBorn?: number;\n    fiscalYear: number;\n    totalPay?: number;\n    exercisedValue: number;\n    unexercisedValue: number;\n  }[];\n  auditRisk: number;\n  boardRisk: number;\n  compensationRisk: number;\n  shareHolderRightsRisk: number;\n  overallRisk: number;\n  governanceEpochDate: number;\n  compensationAsOfEpochDate: number;\n  irWebsite: string;\n  maxAge: number;\n  priceHint: number;\n  previousClose: number;\n  open: number;\n  dayLow: number;\n  dayHigh: number;\n  regularMarketPreviousClose: number;\n  regularMarketOpen: number;\n  regularMarketDayLow: number;\n  regularMarketDayHigh: number;\n  dividendRate: number;\n  dividendYield: number;\n  exDividendDate: number;\n  payoutRatio: number;\n  fiveYearAvgDividendYield: number;\n  beta: number;\n  trailingPE: number;\n  forwardPE: number;\n  volume: number;\n  regularMarketVolume: number;\n  averageVolume: number;\n  averageVolume10days: number;\n  averageDailyVolume10Day: number;\n  bid: number;\n  ask: number;\n  bidSize: number;\n  askSize: number;\n  marketCap: number;\n  fiftyTwoWeekLow: number;\n  fiftyTwoWeekHigh: number;\n  priceToSalesTrailing12Months: number;\n  fiftyDayAverage: number;\n  twoHundredDayAverage: number;\n  trailingAnnualDividendRate: number;\n  trailingAnnualDividendYield: number;\n  currency: string;\n  enterpriseValue: number;\n  profitMargins: number;\n  floatShares: number;\n  sharesOutstanding: number;\n  sharesShort: number;\n  sharesShortPriorMonth: number;\n  sharesShortPreviousMonthDate: number;\n  dateShortInterest: number;\n  sharesPercentSharesOut: number;\n  heldPercentInsiders: number;\n  heldPercentInstitutions: number;\n  shortRatio: number;\n  shortPercentOfFloat: number;\n  impliedSharesOutstanding: number;\n  bookValue: number;\n  priceToBook: number;\n  lastFiscalYearEnd: number;\n  nextFiscalYearEnd: number;\n  mostRecentQuarter: number;\n  earningsQuarterlyGrowth: number;\n  netIncomeToCommon: number;\n  trailingEps: number;\n  forwardEps: number;\n  pegRatio: number;\n  lastSplitFactor: string;\n  lastSplitDate: number;\n  enterpriseToRevenue: number;\n  enterpriseToEbitda: number;\n  \'52WeekChange\': number;\n  \'SandP52WeekChange\': number;\n  lastDividendValue: number;\n  lastDividendDate: number;\n  exchange: string;\n  quoteType: string;\n  symbol: string;\n  underlyingSymbol: string;\n  shortName: string;\n  longName: string;\n  firstTradeDateEpochUtc: number;\n  timeZoneFullName: string;\n  timeZoneShortName: string;\n  uuid: string;\n  messageBoardId: string;\n  gmtOffSetMilliseconds: number;\n  currentPrice: number;\n  targetHighPrice: number;\n  targetLowPrice: number;\n  targetMeanPrice: number;\n  targetMedianPrice: number;\n  recommendationMean: number;\n  recommendationKey: string;\n  numberOfAnalystOpinions: number;\n  totalCash: number;\n  totalCashPerShare: number;\n  ebitda: number;\n  totalDebt: number;\n  quickRatio: number;\n  currentRatio: number;\n  totalRevenue: number;\n  debtToEquity: number;\n  revenuePerShare: number;\n  returnOnAssets: number;\n  returnOnEquity: number;\n  freeCashflow: number;\n  operatingCashflow: number;\n  earningsGrowth: number;\n  revenueGrowth: number;\n  grossMargins: number;\n  ebitdaMargins: number;\n  operatingMargins: number;\n  financialCurrency: string;\n  trailingPegRatio: number;\n};\n\ntype PriceData = {\n  Open: { [timestamp: string]: number };\n  High: { [timestamp: string]: number };\n  Low: { [timestamp: string]: number };\n  Close: { [timestamp: string]: number };\n  Volume: { [timestamp: string]: number };\n  Dividends: { [timestamp: string]: number };\n  "Stock Splits": { [timestamp: string]: number };\n};\n```\n\nThe commands to seek information are:\n\n| Command | Arguments | Description | Output Format |\n| --- | --- | --- | --- |\n| get_info | ticker | Retrieves stock information from Yahoo Finance | Info |\n| get_history | ticker, period, interval, start, end | Retrieves stock history from Yahoo Finance | PriceData |\n\nThe optional input types are:\n- period: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max\n- interval: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo\n- start: YYYY-MM-DD\n- end: YYYY-MM-DD\nYou may omit an optional input type if it is not needed. For period and interval the explicit values from the list must be used, for example you cannot use 1wk for the period.\n\nYour response must be in pure JSON format without any additional markers or formatting. The JSON will include a "Thought" + "Action" to retrieve data that you need in order to answer the question, or it will include the "Answer". When data has been retrieved, it will be included as an "Observation".\n\nYou will continue generating thoughts and actions until you get to an answer, or conclude that you can\'t.\n\nIMPORTANT: Ensure that your entire response is valid JSON. Do not include any text outside of the JSON structure, and do not use code block markers or language identifiers. The answer field in the JSON must be a descriptive string, it should not be a JSON.\n\nYou may take as many steps as necessary.\nHere are some examples.\n\nExample 0:\n```\nUser: Who is the CEO of MSFT?\n\nAssistant: {\n  "thought": "Yahoo Finance likely has this information. I\'m looking it up...",\n  "action": {"command": "get_info", "ticker": "MSFT"}\n}\n\nAssistant: {\n  "observation": {...,"companyOfficers":[{"maxAge":1,"name":"Mr. Satya  Nadella","age":56,"title":"Chairman & CEO","yearBorn":1967,"fiscalYear":2023,"totalPay":9276400,"exercisedValue":0,"unexercisedValue":0},...],...}\n}\n\nAssistant: {\n  "answer": "The CEO of MSFT is Satya Nadella."\n}\n\nReflection: I directly retrieved MSFT\'s company information and answered with the CEO\'s name. I could have provided more context by explaining the data source and offering additional details about Satya Nadella\'s tenure or achievements as CEO, if the user wanted more comprehensive information.\n```\n\nExample 1:\n```\nUser: What was AAPL\'s highest stock price in the past month, and how does it compare to their 52-week high?\n\nAssistant: {\n  "thought": "To answer this question, I need to first get the current stock information for AAPL, including its 52-week high. Then, I\'ll need to retrieve the stock price history for the past month to find the highest price during that period.",\n  "action": {"command": "get_info", "ticker": "AAPL"}\n}\n\nAssistant: {\n  "observation": {"symbol": "AAPL", "shortName": "Apple Inc.", "fiftyTwoWeekHigh": 198.23, "currency": "USD", ...}\n}\n\nAssistant: {\n  "thought": "I\'ve got the current stock information for AAPL, including the 52-week high of $198.23. Now I need to get the stock price history for the past month to find the highest price during that period.",\n  "action": {"command": "get_history", "ticker": "AAPL", "period": "1mo", "interval": "1d"}\n}\n\nAssistant: {\n  "observation": {\n    "High": {\n      "1688407800": 192.67,\n      "1688494200": 189.99,\n      ...,\n      "1690912200": 195.71\n    },\n    ...\n  }\n}\n\nAssistant: {\n  "thought": "I now have all the information needed to answer the question. Let me analyze the data and provide the answer.",\n  "answer": "AAPL\'s highest stock price in the past month was $195.71. This is 1.27% lower than their 52-week high of $198.23. The highest price in the past month occurred on August 1, 2023 (assuming the last date in the data is the most recent), and it came close to, but did not exceed, the 52-week high."\n}\n\nReflection: I retrieved AAPL\'s current stock information and 52-week high, then fetched the past month\'s price history. This two-step approach was thorough, but I could have offered to provide more context about the factors influencing AAPL\'s recent stock performance if the user desired more in-depth analysis.\n```\n\n(END OF EXAMPLES)\n',
    },
    {
        "role": "user",
        "content": "What was NVDA's highest stock price in the past month, and how does it compare to their 52-week high",
    },
    {
        "role": "assistant",
        "content": '{\n  "thought": "To answer this question, I need to first get the current stock information for NVDA, including its 52-week high. Then, I\'ll need to retrieve the stock price history for the past month to find the highest price during that period.",\n  "action": {"command": "get_info", "ticker": "NVDA"}\n}',
    },
    {
        "role": "assistant",
        "content": '{"observation": {"address1": "2788 San Tomas Expressway", "city": "Santa Clara", "state": "CA", "zip": "95051", "country": "United States", "phone": "408 486 2000", "website": "https://www.nvidia.com", "industry": "Semiconductors", "industryKey": "semiconductors", "industryDisp": "Semiconductors", "sector": "Technology", "sectorKey": "technology", "sectorDisp": "Technology", "longBusinessSummary": "NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; virtual GPU or vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software for building and operating metaverse and 3D internet applications. The Compute & Networking segment comprises Data Center computing platforms and end-to-end networking platforms, including Quantum for InfiniBand and Spectrum for Ethernet; NVIDIA DRIVE automated-driving platform and automotive development agreements; Jetson robotics and other embedded platforms; NVIDIA AI Enterprise and other software; and DGX Cloud software and services. The company\'s products are used in gaming, professional visualization, data center, and automotive markets. It sells its products to original equipment manufacturers, original device manufacturers, system integrators and distributors, independent software vendors, cloud service providers, consumer internet companies, add-in board manufacturers, distributors, automotive manufacturers and tier-1 automotive suppliers, and other ecosystem participants. NVIDIA Corporation was incorporated in 1993 and is headquartered in Santa Clara, California.", "fullTimeEmployees": 29600, "companyOfficers": [{"maxAge": 1, "name": "Mr. Jen-Hsun  Huang", "age": 60, "title": "Co-Founder, CEO, President & Director", "yearBorn": 1963, "fiscalYear": 2024, "totalPay": 7491487, "exercisedValue": 217327152, "unexercisedValue": 0}, {"maxAge": 1, "name": "Ms. Colette M. Kress", "age": 56, "title": "Executive VP & CFO", "yearBorn": 1967, "fiscalYear": 2024, "totalPay": 1510765, "exercisedValue": 0, "unexercisedValue": 0}, {"maxAge": 1, "name": "Ms. Debora  Shoquist", "age": 68, "title": "Executive Vice President of Operations", "yearBorn": 1955, "fiscalYear": 2024, "totalPay": 1371266, "exercisedValue": 0, "unexercisedValue": 0}, {"maxAge": 1, "name": "Mr. Timothy S. Teter", "age": 56, "title": "Executive VP, General Counsel & Secretary", "yearBorn": 1967, "fiscalYear": 2024, "totalPay": 1360939, "exercisedValue": 0, "unexercisedValue": 0}, {"maxAge": 1, "name": "Mr. Ajay K. Puri", "age": 68, "title": "Executive Vice President of Worldwide Field Operations", "yearBorn": 1955, "fiscalYear": 2024, "totalPay": 2295097, "exercisedValue": 0, "unexercisedValue": 0}, {"maxAge": 1, "name": "Mr. Chris A. Malachowsky", "title": "Co-Founder", "fiscalYear": 2024, "totalPay": 320000, "exercisedValue": 0, "unexercisedValue": 0}, {"maxAge": 1, "name": "Mr. Donald F. Robertson Jr.", "age": 54, "title": "VP & Chief Accounting Officer", "yearBorn": 1969, "fiscalYear": 2024, "exercisedValue": 0, "unexercisedValue": 0}, {"maxAge": 1, "name": "Prof. William J. Dally Ph.D.", "age": 62, "title": "Chief Scientist & Senior VP of Research", "yearBorn": 1961, "fiscalYear": 2024, "exercisedValue": 0, "unexercisedValue": 0}, {"maxAge": 1, "name": "Ms. Mylene  Mangalindan", "title": "VP of Corporate Communications", "fiscalYear": 2024, "exercisedValue": 0, "unexercisedValue": 0}, {"maxAge": 1, "name": "Mr. Tommy  Lee", "title": "Senior Vice President of Systems Engineering & Application", "fiscalYear": 2024, "exercisedValue": 0, "unexercisedValue": 0}], "auditRisk": 7, "boardRisk": 10, "compensationRisk": 6, "shareHolderRightsRisk": 6, "overallRisk": 8, "governanceEpochDate": 1719792000, "compensationAsOfEpochDate": 1735603200, "irWebsite": "http://phx.corporate-ir.net/phoenix.zhtml?c=116466&p=irol-IRHome", "maxAge": 86400, "priceHint": 2, "previousClose": 112.28, "open": 116.15, "dayLow": 111.58, "dayHigh": 116.2, "regularMarketPreviousClose": 112.28, "regularMarketOpen": 116.15, "regularMarketDayLow": 111.58, "regularMarketDayHigh": 116.2, "dividendRate": 0.04, "dividendYield": 0.0004, "exDividendDate": 1718064000, "payoutRatio": 0.0094, "fiveYearAvgDividendYield": 0.12, "beta": 1.68, "trailingPE": 66.11696, "forwardPE": 30.229946, "volume": 293399073, "regularMarketVolume": 293399073, "averageVolume": 371326731, "averageVolume10days": 286404060, "averageDailyVolume10Day": 286404060, "bid": 111.71, "ask": 113.1, "bidSize": 100, "askSize": 200, "marketCap": 2781083860992, "fiftyTwoWeekLow": 39.23, "fiftyTwoWeekHigh": 140.76, "priceToSalesTrailing12Months": 34.862034, "fiftyDayAverage": 118.85288, "twoHundredDayAverage": 78.96024, "trailingAnnualDividendRate": 0.016, "trailingAnnualDividendYield": 0.0001425009, "currency": "USD", "enterpriseValue": 2761074933760, "profitMargins": 0.53398, "floatShares": 23605422000, "sharesOutstanding": 24598300672, "sharesShort": 280295880, "sharesShortPriorMonth": 315033985, "sharesShortPreviousMonthDate": 1718323200, "dateShortInterest": 1721001600, "sharesPercentSharesOut": 0.0114, "heldPercentInsiders": 0.042930003, "heldPercentInstitutions": 0.67279, "shortRatio": 0.87, "shortPercentOfFloat": 0.0119, "impliedSharesOutstanding": 25016600576, "bookValue": 1.998, "priceToBook": 56.586586, "lastFiscalYearEnd": 1706400000, "nextFiscalYearEnd": 1738022400, "mostRecentQuarter": 1714262400, "earningsQuarterlyGrowth": 6.284, "netIncomeToCommon": 42597998592, "trailingEps": 1.71, "forwardEps": 3.74, "pegRatio": 0.9, "lastSplitFactor": "10:1", "lastSplitDate": 1717977600, "enterpriseToRevenue": 34.611, "enterpriseToEbitda": 56.034, "52WeekChange": 1.4194825, "SandP52WeekChange": 0.18961596, "lastDividendValue": 0.01, "lastDividendDate": 1718064000, "exchange": "NMS", "quoteType": "EQUITY", "symbol": "NVDA", "underlyingSymbol": "NVDA", "shortName": "NVIDIA Corporation", "longName": "NVIDIA Corporation", "firstTradeDateEpochUtc": 917015400, "timeZoneFullName": "America/New_York", "timeZoneShortName": "EDT", "uuid": "7f5f6a07-b148-30f4-98a2-2caa3df2aed0", "messageBoardId": "finmb_32307", "gmtOffSetMilliseconds": -14400000, "currentPrice": 113.06, "targetHighPrice": 200.0, "targetLowPrice": 50.3, "targetMeanPrice": 135.95, "targetMedianPrice": 140.5, "recommendationMean": 1.8, "recommendationKey": "buy", "numberOfAnalystOpinions": 48, "totalCash": 31438000128, "totalCashPerShare": 1.278, "ebitda": 49274998784, "totalDebt": 11237000192, "quickRatio": 2.877, "currentRatio": 3.529, "totalRevenue": 79773999104, "debtToEquity": 22.866, "revenuePerShare": 3.234, "returnOnAssets": 0.49103, "returnOnEquity": 1.15658, "freeCashflow": 29023750144, "operatingCashflow": 40524001280, "earningsGrowth": 6.5, "revenueGrowth": 2.621, "grossMargins": 0.75286, "ebitdaMargins": 0.61768, "operatingMargins": 0.64925003, "financialCurrency": "USD", "trailingPegRatio": 1.2843}}',
    },
    {
        "role": "assistant",
        "content": '{\n  "thought": "I\'ve got the current stock information for NVDA, including the 52-week high of $140.76. Now I need to get the stock price history for the past month to find the highest price during that period.",\n  "action": {"command": "get_history", "ticker": "NVDA", "period": "1mo", "interval": "1d"}\n}',
    },
    {
        "role": "assistant",
        "content": '{"observation": "{\\"Open\\":{\\"1719460800000\\":124.0999984741,\\"1719547200000\\":124.5800018311,\\"1719806400000\\":123.4700012207,\\"1719892800000\\":121.1299972534,\\"1719979200000\\":121.6600036621,\\"1720152000000\\":127.3799972534,\\"1720411200000\\":127.4899978638,\\"1720497600000\\":130.3500061035,\\"1720584000000\\":134.0299987793,\\"1720670400000\\":135.75,\\"1720756800000\\":128.2599945068,\\"1721016000000\\":130.5599975586,\\"1721102400000\\":128.4400024414,\\"1721188800000\\":121.3499984741,\\"1721275200000\\":121.8499984741,\\"1721361600000\\":120.3499984741,\\"1721620800000\\":120.3499984741,\\"1721707200000\\":122.7799987793,\\"1721793600000\\":119.1699981689,\\"1721880000000\\":113.0400009155,\\"1721966400000\\":116.1900024414},\\"High\\":{\\"1719460800000\\":126.4100036621,\\"1719547200000\\":127.7099990845,\\"1719806400000\\":124.8399963379,\\"1719892800000\\":123.4100036621,\\"1719979200000\\":128.2799987793,\\"1720152000000\\":128.8500061035,\\"1720411200000\\":130.7700042725,\\"1720497600000\\":133.8200073242,\\"1720584000000\\":135.1000061035,\\"1720670400000\\":136.1499938965,\\"1720756800000\\":131.9199981689,\\"1721016000000\\":131.3899993896,\\"1721102400000\\":129.0399932861,\\"1721188800000\\":121.8499984741,\\"1721275200000\\":122.4000015259,\\"1721361600000\\":121.5999984741,\\"1721620800000\\":124.0699996948,\\"1721707200000\\":124.6900024414,\\"1721793600000\\":119.9499969482,\\"1721880000000\\":116.6299972534,\\"1721966400000\\":116.1999969482},\\"Low\\":{\\"1719460800000\\":122.9199981689,\\"1719547200000\\":122.75,\\"1719806400000\\":118.8300018311,\\"1719892800000\\":121.0299987793,\\"1719979200000\\":121.3600006104,\\"1720152000000\\":125.6800003052,\\"1720411200000\\":127.0400009155,\\"1720497600000\\":128.6499938965,\\"1720584000000\\":132.4199981689,\\"1720670400000\\":127.0500030518,\\"1720756800000\\":127.2200012207,\\"1721016000000\\":127.1800003052,\\"1721102400000\\":124.5800018311,\\"1721188800000\\":116.7200012207,\\"1721275200000\\":116.5599975586,\\"1721361600000\\":117.3700027466,\\"1721620800000\\":119.8600006104,\\"1721707200000\\":122.0999984741,\\"1721793600000\\":113.4400024414,\\"1721880000000\\":106.3000030518,\\"1721966400000\\":111.5800018311},\\"Close\\":{\\"1719460800000\\":123.9899978638,\\"1719547200000\\":123.5400009155,\\"1719806400000\\":124.3000030518,\\"1719892800000\\":122.6699981689,\\"1719979200000\\":128.2799987793,\\"1720152000000\\":125.8300018311,\\"1720411200000\\":128.1999969482,\\"1720497600000\\":131.3800048828,\\"1720584000000\\":134.9100036621,\\"1720670400000\\":127.4000015259,\\"1720756800000\\":129.2400054932,\\"1721016000000\\":128.4400024414,\\"1721102400000\\":126.3600006104,\\"1721188800000\\":117.9899978638,\\"1721275200000\\":121.0899963379,\\"1721361600000\\":117.9300003052,\\"1721620800000\\":123.5400009155,\\"1721707200000\\":122.5899963379,\\"1721793600000\\":114.25,\\"1721880000000\\":112.2799987793,\\"1721966400000\\":113.0599975586},\\"Volume\\":{\\"1719460800000\\":252571700,\\"1719547200000\\":315516700,\\"1719806400000\\":284885500,\\"1719892800000\\":218374000,\\"1719979200000\\":215749000,\\"1720152000000\\":214176700,\\"1720411200000\\":237677300,\\"1720497600000\\":285366600,\\"1720584000000\\":248978600,\\"1720670400000\\":374782700,\\"1720756800000\\":252680500,\\"1721016000000\\":208326200,\\"1721102400000\\":214769500,\\"1721188800000\\":390086200,\\"1721275200000\\":320979500,\\"1721361600000\\":217223800,\\"1721620800000\\":258068900,\\"1721707200000\\":173911000,\\"1721793600000\\":327776900,\\"1721880000000\\":460067000,\\"1721966400000\\":292831600},\\"Dividends\\":{\\"1719460800000\\":0.0,\\"1719547200000\\":0.0,\\"1719806400000\\":0.0,\\"1719892800000\\":0.0,\\"1719979200000\\":0.0,\\"1720152000000\\":0.0,\\"1720411200000\\":0.0,\\"1720497600000\\":0.0,\\"1720584000000\\":0.0,\\"1720670400000\\":0.0,\\"1720756800000\\":0.0,\\"1721016000000\\":0.0,\\"1721102400000\\":0.0,\\"1721188800000\\":0.0,\\"1721275200000\\":0.0,\\"1721361600000\\":0.0,\\"1721620800000\\":0.0,\\"1721707200000\\":0.0,\\"1721793600000\\":0.0,\\"1721880000000\\":0.0,\\"1721966400000\\":0.0},\\"Stock Splits\\":{\\"1719460800000\\":0.0,\\"1719547200000\\":0.0,\\"1719806400000\\":0.0,\\"1719892800000\\":0.0,\\"1719979200000\\":0.0,\\"1720152000000\\":0.0,\\"1720411200000\\":0.0,\\"1720497600000\\":0.0,\\"1720584000000\\":0.0,\\"1720670400000\\":0.0,\\"1720756800000\\":0.0,\\"1721016000000\\":0.0,\\"1721102400000\\":0.0,\\"1721188800000\\":0.0,\\"1721275200000\\":0.0,\\"1721361600000\\":0.0,\\"1721620800000\\":0.0,\\"1721707200000\\":0.0,\\"1721793600000\\":0.0,\\"1721880000000\\":0.0,\\"1721966400000\\":0.0}}"}',
    },
    {
        "role": "assistant",
        "content": '{\n  "thought": "I now have all the information needed to answer the question. Let me analyze the data and provide the answer.",\n  "answer": "NVDA\'s highest stock price in the past month was $137.42. This occurred on July 10, 2024. This price is 2.37% lower than their 52-week high of $140.76."\n}',
    },
]


def format_trial(trial):
    formatted_trial = "User: " + trial[1]["content"] + "\n\n"

    for message in trial[2:]:
        if message["role"] == "assistant":
            content = eval(message["content"])
            formatted_trial += "Assistant: {\n"
            if "thought" in content:
                formatted_trial += f'  "thought": "{content["thought"]}",\n'
                if "action" in content:
                    formatted_trial += f'  "action": {json.dumps(content["action"])}\n'
            elif "observation" in content:
                formatted_trial += f'  "observation": {content["observation"]}\n'
            if "answer" in content:
                formatted_trial += f'  "answer": "{content["answer"]}"\n'
            formatted_trial += "}\n\n"

    formatted_trial += (
        "Assistant: {\n" + '  "observation": "Answer is INCORRECT"\n' + "}\n\n"
    )

    return formatted_trial.strip()


example_formatted_trial = format_trial(example_previous_trial)
print(example_formatted_trial)

User: What was NVDA's highest stock price in the past month, and how does it compare to their 52-week high

Assistant: {
  "thought": "To answer this question, I need to first get the current stock information for NVDA, including its 52-week high. Then, I'll need to retrieve the stock price history for the past month to find the highest price during that period.",
  "action": {"command": "get_info", "ticker": "NVDA"}
}

Assistant: {
  "observation": {'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '408 486 2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryKey': 'semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The Graphics segm

In [143]:
self_reflection_system_prompt = f"""You are an advanced reasoning agent that can improve based on self refection. You will be given a previous reasoning trial in which you were given access to an Docstore API environment and a question to answer. You were unsuccessful in answering the question either because you guessed the wrong answer with Finish[], or you used up your set number of reasoning steps. In a few sentences, Diagnose a possible reason for failure and devise a new, concise, high level plan that aims to mitigate the same failure. Use complete sentences.

Here are some examples.

{examples}

(END OF EXAMPLES)"""

self_reflection_system_tokens = len(tokenizer.encode(self_reflection_system_prompt))
print("Tokens:", self_reflection_system_tokens)
print()
print(self_reflection_system_prompt)

Tokens: 800

You are an advanced reasoning agent that can improve based on self refection. You will be given a previous reasoning trial in which you were given access to an Docstore API environment and a question to answer. You were unsuccessful in answering the question either because you guessed the wrong answer with Finish[], or you used up your set number of reasoning steps. In a few sentences, Diagnose a possible reason for failure and devise a new, concise, high level plan that aims to mitigate the same failure. Use complete sentences.

Here are some examples.

Example 0:
```
User: Who is the CEO of MSFT?

Assistant: {
  "thought": "Yahoo Finance likely has this information. I'm looking it up...",
  "action": {"command": "get_info", "ticker": "MSFT"}
}

Assistant: {
  "observation": {...,"companyOfficers":[{"maxAge":1,"name":"Mr. Satya  Nadella","age":56,"title":"Chairman & CEO","yearBorn":1967,"fiscalYear":2023,"totalPay":9276400,"exercisedValue":0,"unexercisedValue":0},...],...

In [144]:
def self_reflection_prompt(previous_trial: str):
    return f"""Previous trial:
{previous_trial}

Reflection:"""


self_reflection_tokens = len(
    tokenizer.encode(self_reflection_prompt(example_formatted_trial))
)
print("Tokens:", self_reflection_tokens)
print()
print(self_reflection_prompt(example_formatted_trial))

Tokens: 4418

Previous trial:
User: What was NVDA's highest stock price in the past month, and how does it compare to their 52-week high

Assistant: {
  "thought": "To answer this question, I need to first get the current stock information for NVDA, including its 52-week high. Then, I'll need to retrieve the stock price history for the past month to find the highest price during that period.",
  "action": {"command": "get_info", "ticker": "NVDA"}
}

Assistant: {
  "observation": {'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '408 486 2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryKey': 'semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and inte

In [145]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": self_reflection_system_prompt},
        {"role": "user", "content": self_reflection_prompt(example_formatted_trial)},
    ],
)

print(response.choices[0].message.content)

The failure in the previous attempt was due to an incorrect extraction and analysis of the data for NVDA's highest stock price in the past month. I misinterpreted the information, leading to an incorrect conclusion.

**New Plan:**
1. Retrieve the current stock information for NVDA, including the 52-week high.
2. Get the stock price history for the past month.
3. Correctly identify the highest closing price from the historical data by ensuring that I examine the "High" values correctly.
4. Compare this identified highest price with the 52-week high and provide this analysis clearly.
5. Revalidate the result before concluding.

Using a more meticulous data extraction method will help ensure accuracy.


In [146]:
def get_info(ticker: str):
    stock = yf.Ticker(ticker)
    return stock.info

In [147]:
def get_history(
    ticker: str,
    period="1mo",
    interval="1d",
    start=None,
    end=None,
):
    """
    :Parameters:
        period : str
            Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            Either Use period parameter or use start and end
        interval : str
            Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            Intraday data cannot extend last 60 days
        start: str
            Download start date string (YYYY-MM-DD) or _datetime, inclusive.
            Default is 99 years ago
            E.g. for start="2020-01-01", the first data point will be on "2020-01-01"
        end: str
            Download end date string (YYYY-MM-DD) or _datetime, exclusive.
            Default is now
            E.g. for end="2023-01-01", the last data point will be on "2022-12-31"
    """
    stock = yf.Ticker(ticker)
    return stock.history(
        period=period, interval=interval, start=start, end=end
    ).to_json()

In [148]:
def act(message: str, debug=False) -> tuple[dict, str]:
    if message is None or message == "":
        return ({"error": "No message provided"}, "")

    data = {}
    try:
        data = json.loads(message)
    except Exception as e:
        if debug:
            print(message)
        return ({"error": str(e)}, "")

    if debug:
        print("Assistant:")
        print(data)

    if "answer" in data:
        return ({}, data["answer"])

    if "action" not in data:
        return ({"error": "No action provided"}, "")

    action = data["action"]

    if action["command"] == "get_info":
        if "ticker" not in action:
            return ({"error": "No ticker provided"}, "")

        ticker = action["ticker"]

        try:
            return (get_info(ticker), "")
        except Exception as e:
            return ({"error": str(e)}, "")
    elif action["command"] == "get_history":
        if "ticker" not in action:
            return ({"error": "No ticker provided"}, "")

        ticker = action["ticker"]  # This should be required, so keep it as is
        period = action.get("period", "1mo")  # Default to "1mo" if not provided
        interval = action.get("interval", "1d")  # Default to "1d" if not provided
        start = action.get("start")  # Will be None if not provided
        end = action.get("end")  # Will be None if not provided

        try:
            return (get_history(ticker, period, interval, start, end), "")
        except Exception as e:
            return ({"error": str(e)}, "")

    return ({"error": "Unknown command"}, "")

In [149]:
def loop(query: str, expected_answer: str, debug=False):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": query},
    ]
    tokens = system_message_tokens + len(tokenizer.encode(query))

    while True:
        while tokens > CONTEXT_WINDOW:
            for i, msg in enumerate(messages):
                if msg["role"] == "assistant":
                    removed_msg = messages.pop(i)
                    truncated_content = (
                        removed_msg["content"][:100] + "..."
                        if len(removed_msg["content"]) > 100
                        else removed_msg["content"]
                    )
                    print("Pruning message:", truncated_content)

                    tokens -= len(tokenizer.encode(removed_msg["content"]))
                    break

        if debug:
            print("Tokens:", tokens)

        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
        )

        assistant_message = response.choices[0].message.content
        messages.append({"role": "assistant", "content": assistant_message})
        tokens += len(tokenizer.encode(assistant_message))

        observation, answer = act(assistant_message, debug)
        if answer:
            return answer

        if debug:
            print("Observation:")
            print_observation = json.dumps(observation)
            truncated_observation = (
                print_observation[:100] + "..."
                if len(print_observation) > 100
                else print_observation
            )
            print(truncated_observation)

        observation_content = json.dumps({"observation": observation})
        messages.append({"role": "assistant", "content": observation_content})
        tokens += len(tokenizer.encode(observation_content))

In [150]:
answer = loop("Who is the CEO of NVDA?", "Jen-Hsun Huang", True)
print()
print("Answer:", answer)

Tokens: 2155
Assistant:
{'thought': 'I need to retrieve the company information for NVDA to find out the CEO.', 'action': {'command': 'get_info', 'ticker': 'NVDA'}}
Observation:
{"address1": "2788 San Tomas Expressway", "city": "Santa Clara", "state": "CA", "zip": "95051", "cou...
Tokens: 4576
Assistant:
{'answer': 'The CEO of NVDA is Jen-Hsun Huang, who is also a co-founder and the President of the company.'}

Answer: The CEO of NVDA is Jen-Hsun Huang, who is also a co-founder and the President of the company.


In [151]:
answer = loop(
    "What was NVDA's highest stock price in the past month, and how does it compare to their 52-week high",
    "3.27%",
    True,
)
print()
print("Answer:", answer)

Tokens: 2171
Assistant:
{'thought': "To answer this question, I need to first get the current stock information for NVDA, including its 52-week high. Then, I'll need to retrieve the stock price history for the past month to find the highest price during that period.", 'action': {'command': 'get_info', 'ticker': 'NVDA'}}
Observation:
{"address1": "2788 San Tomas Expressway", "city": "Santa Clara", "state": "CA", "zip": "95051", "cou...
Tokens: 4625
Assistant:
{'thought': "I've got the current stock information for NVDA, including the 52-week high of $140.76. Now I need to get the stock price history for the past month to find the highest price during that period.", 'action': {'command': 'get_history', 'ticker': 'NVDA', 'period': '1mo', 'interval': '1d'}}
Observation:
"{\"Open\":{\"1719806400000\":123.4700012207,\"1719892800000\":121.1299972534,\"1719979200000\":121....
Tokens: 6372
Assistant:
{'thought': 'I now have all the information needed to answer the question. Let me analyze the d

In [152]:
answer = loop(
    "What was the price of NVDA 2 years back and how does it compare to now?",
    "17.96",
    True,
)
print()
print("Answer:", answer)

Tokens: 2166
Assistant:
{'thought': 'To answer this question, I need to first get the current stock information for NVDA. Then, I will retrieve the stock price history for the date two years back (July 30, 2022) to make a comparison.', 'action': {'command': 'get_info', 'ticker': 'NVDA'}}
Observation:
{"address1": "2788 San Tomas Expressway", "city": "Santa Clara", "state": "CA", "zip": "95051", "cou...
Tokens: 4618


$NVDA: possibly delisted; No price data found  (1d 2022-07-30 -> 2022-07-30)


Assistant:
{'thought': 'I have the current price of NVDA which is $111.59. Now, I need to retrieve the stock price history for NVDA from two years back (July 30, 2022).', 'action': {'command': 'get_history', 'ticker': 'NVDA', 'period': '1d', 'interval': '1d', 'start': '2022-07-30', 'end': '2022-07-30'}}
$NVDA: possibly delisted; No price data found  (1d 2022-07-30 -> 2022-07-30)
Observation:
"{\"Open\":{},\"High\":{},\"Low\":{},\"Close\":{},\"Adj Close\":{},\"Volume\":{}}"
Tokens: 4760
Assistant:
{'thought': 'The stock price history for NVDA on July 30, 2022, is not available. This could be due to a non-trading day (such as a weekend or holiday). I will check the nearest available date before July 30, 2022.', 'action': {'command': 'get_history', 'ticker': 'NVDA', 'period': '5d', 'interval': '1d', 'start': '2022-07-25', 'end': '2022-07-29'}}
Observation:
"{\"Open\":{\"1658721600000\":16.999854945,\"1658808000000\":16.870002855,\"1658894400000\":17.01284...
Tokens: 5239
Assistant:
{'answ